In [1]:
from gensim.test.utils import datapath
from gensim.models import KeyedVectors
wv_from_bin = KeyedVectors.load_word2vec_format(datapath("/Users/raunakpandey/Downloads/word2vec/model.bin"), binary=True)

In [2]:
word2vec_model = wv_from_bin

In [3]:
import numpy as np
import spacy
# Load spaCy model for part-of-speech tagging
nlp = spacy.load("en_core_web_sm")

In [4]:
import pandas as pd
df = pd.read_csv("/Users/raunakpandey/Documents/programming/projects/augmentation/Augmentations.csv")

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317 entries, 0 to 316
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      317 non-null    int64 
 1   premise         317 non-null    object
 2   label           317 non-null    int64 
 3   swap_sub_obj    317 non-null    object
 4   negation        317 non-null    object
 5   numbers_change  317 non-null    object
dtypes: int64(2), object(4)
memory usage: 15.0+ KB


## noun

In [6]:

def find_most_different_noun(word):
    try:
        # Get the Word2Vec embedding for the given word
        word_embedding = word2vec_model[word+'_NOUN']
        # print(word_embedding)
        # Find the most contextually different verb
        most_different_noun = None
        max_similarity = float(0)

        for noun in word2vec_model.index_to_key:
            if noun.endswith("_NOUN"):  # Consider only noun
                # print(noun)
                noun_embedding = word2vec_model[noun]
                similarity = word2vec_model.cosine_similarities(word_embedding, [noun_embedding])[0]
                # print(similarity)
                
                if similarity > max_similarity and similarity < 0.9:
                    most_different_noun = noun
                    max_similarity = similarity

        return most_different_noun.split("_")[0]
    except KeyError:
        return None

In [7]:
print(find_most_different_noun("pink"))

cerise


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def choose_most_dissimilar_sentence(modified_sentence, sentence):
    # Create TF-IDF vectors for all sentences
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(modified_sentence+[sentence])

    # Calculate cosine similarity between the given sentence and all sentences in the list
    cosine_similarities = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])

    # Find the sentence with the lowest cosine similarity (most dissimilar)
    most_dissimilar_index = cosine_similarities.argmin()

    # Randomly choose one of the most dissimilar sentences
    random_dissimilar_sentence = modified_sentence[most_dissimilar_index]

    return random_dissimilar_sentence


In [9]:
def choose_first_two(modified_sentence, sentence):
    ret1 = sentence
    ret2 = sentence
    for i in modified_sentence:
        if i != sentence:
            ret1 = i
            break
    for i in modified_sentence:
        if i != sentence and i != ret1:
            ret2 = i
            break
    return ret1,ret2

In [10]:
def choose_sent(modified_sentence, sentence):
    ret_sent = modified_sentence[0][0]
    similarity = modified_sentence[0][1]
    for sent, sim in modified_sentence:
        if sent != sentence and sim < similarity:
            ret_sent = sent
            similarity = sim
            break
    return ret_sent

In [11]:
def replace_noun_with_most_different(sentence):
    from tqdm.auto import tqdm
    tqdm.pandas()
    
    # Process the sentence with spaCy
    doc = nlp(sentence)

    # Find a verb in the sentence
    target_noun = []
    for token in doc:
        if token.pos_ == "PROPN" or token.pos_ == "NOUN":
            target_noun.append(token.text)
    # print(target_noun)
    if target_noun is None:
        return sentence

    # Find the contextually most different noun
    modified_sentence = []
    for i in range(len(target_noun)):
        most_different_noun = find_most_different_noun(target_noun[i])
        # print(most_different_noun)
        if most_different_noun is None:
            continue
        new_sentence = sentence.replace(target_noun[i], most_different_noun)
        modified_sentence.append(new_sentence)
    # print(sentence)
    # print(choose_first_two(modified_sentence,sentence))
    # print("--------------------")
    return choose_first_two(modified_sentence,sentence)

In [12]:
# Example sentence
original_sentence = "A barista is making a cappuccino at the coffee shop."

# Replace the verb with the contextually most different verb
modified_sentence = replace_noun_with_most_different(original_sentence)

print("Original Sentence:", original_sentence)
print("Modified Sentence:", modified_sentence)

/Users/raunakpandey/Documents/programming/projects/env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Original Sentence: A barista is making a cappuccino at the coffee shop.
Modified Sentence: ('A barista is making a grappa at the coffee shop.', 'A barista is making a cappuccino at the tea shop.')


In [13]:
# Example sentence
original_sentence = "A lifeguard is watching over swimmers at the pool."

# Replace the verb with the contextually most different verb
modified_sentence = replace_noun_with_most_different(original_sentence)

print("Original Sentence:", original_sentence)
print("Modified Sentence:", modified_sentence)

Original Sentence: A lifeguard is watching over swimmers at the pool.
Modified Sentence: ('A lifejacket is watching over swimmers at the pool.', 'A lifeguard is watching over swimmers at the swimming::pool.')


In [14]:
def test(df):
    from tqdm.auto import tqdm
    tqdm.pandas()
    outputs = df['premise'].progress_apply(replace_noun_with_most_different)
    df['change_nouns1'] = outputs.apply(lambda x: x[0])
    df['change_nouns2'] = outputs.apply(lambda x: x[1])
    return df

In [15]:
df = test(df)

  0%|          | 0/317 [00:00<?, ?it/s]

100%|██████████| 317/317 [07:46<00:00,  1.47s/it]


In [16]:
df.head()

,Unnamed: 0,premise,label,swap_sub_obj,negation,numbers_change,change_nouns1,change_nouns2
0,0,The cat is sleeping on the windowsill.,1,The windowsill is sleeping on the cat,The cat is not sleeping on the windowsill .,The cat is sleeping on the windowsill .,The dog is sleeping on the windowsill.,The cat is sleeping on the window-sill.
1,1,A group of friends is playing soccer in the park.,1,A park of friends is playing soccer in the group,A group of friends is not playing soccer in th...,A group of friends is playing soccer in the pa...,A grouping of friends is playing soccer in the...,A group of friends is playing football in the ...
2,2,The chef is preparing a delicious meal in the ...,1,The kitchen is preparing a delicious meal in t...,The chef is not preparing a delicious meal in ...,The chef is preparing a delicious meal in the ...,The cook is preparing a delicious meal in the ...,The chef is preparing a delicious breakfast in...
3,3,A bright rainbow stretches across the sky afte...,1,A bright rain stretches across the sky after t...,A bright rainbow doesn't stretches across the ...,A bright rainbow stretches across the sky afte...,A bright anya stretches across the sky after t...,A bright rainbow stretches across the cloud af...
4,4,The students are listening attentively during ...,1,The lecture are listening attentively during t...,The students are not listening attentively dur...,The students are listening attentively during ...,The students are listening attentively during ...,The students are listening attentively during ...


# verb

In [17]:
def get_verb_root(verb):
    # Use spaCy to get the root form (lemma) of the verb
    doc = nlp(verb)
    for token in doc:
        if token.pos_ == "VERB":
            return token.lemma_
    
    return None

In [18]:
def choose_first(modified_sentence, sentence):
    ret1 = sentence
    for i in modified_sentence:
        if i != sentence:
            ret1 = i
            break
    return ret1

In [19]:
def find_most_different_verb(word):
    word = get_verb_root(word)
    if(word == None):
        return None
    try:
        # Get the Word2Vec embedding for the given word
        word_embedding = word2vec_model[word+'_VERB']
        # print(word_embedding)
        # Find the most contextually different verb
        most_different_verb = None
        max_similarity = float(0)

        for verb in word2vec_model.index_to_key:
            if verb.endswith("_VERB"):  # Consider only adj
                # print(noun)
                verb_embedding = word2vec_model[verb]
                similarity = word2vec_model.cosine_similarities(word_embedding, [verb_embedding])[0]
                # print(similarity)
                
                if similarity > max_similarity and similarity < 0.65:
                    most_different_verb = verb
                    max_similarity = similarity

        return most_different_verb.split("_")[0]
    except KeyError:
        return None

In [20]:
print(find_most_different_verb("running"))

go


In [21]:
def replace_verb_with_most_different(sentence):
    
    # Process the sentence with spaCy
    doc = nlp(sentence)

    # Find a verb in the sentence
    target_verb = []
    for token in doc:
        if token.pos_ == "VERB":
            target_verb.append(token.text)
    # print(target_noun)
    if target_verb is None:
        return sentence

    # Find the contextually most different noun
    modified_sentence = []
    for i in range(len(target_verb)):
        most_different_verb = find_most_different_verb(target_verb[i])
        # print(most_different_noun)
        if most_different_verb is None:
            continue
        new_sentence = sentence.replace(target_verb[i], most_different_verb)
        modified_sentence.append(new_sentence)
    # print(sentence)
    # print(choose_first(modified_sentence,sentence))
    # print("--------------------")
    return choose_first(modified_sentence,sentence)

In [22]:
# Example sentence
original_sentence = "A barista is making a cappuccino at the coffee shop."

# Replace the verb with the contextually most different verb
modified_sentence = replace_verb_with_most_different(original_sentence)

print("Original Sentence:", original_sentence)
print("Modified Sentence:", modified_sentence)

Original Sentence: A barista is making a cappuccino at the coffee shop.
Modified Sentence: A barista is re-balance a cappuccino at the coffee shop.


In [23]:
def test(df):
    from tqdm.auto import tqdm
    tqdm.pandas()
    df['change_verbs'] = df['premise'].progress_apply(replace_verb_with_most_different)
    return df

In [24]:
df = test(df)

100%|██████████| 317/317 [00:44<00:00,  7.12it/s]


In [25]:
df.head()

,Unnamed: 0,premise,label,swap_sub_obj,negation,numbers_change,change_nouns1,change_nouns2,change_verbs
0,0,The cat is sleeping on the windowsill.,1,The windowsill is sleeping on the cat,The cat is not sleeping on the windowsill .,The cat is sleeping on the windowsill .,The dog is sleeping on the windowsill.,The cat is sleeping on the window-sill.,The cat is doze on the windowsill.
1,1,A group of friends is playing soccer in the park.,1,A park of friends is playing soccer in the group,A group of friends is not playing soccer in th...,A group of friends is playing soccer in the pa...,A grouping of friends is playing soccer in the...,A group of friends is playing football in the ...,A group of friends is understudy soccer in the...
2,2,The chef is preparing a delicious meal in the ...,1,The kitchen is preparing a delicious meal in t...,The chef is not preparing a delicious meal in ...,The chef is preparing a delicious meal in the ...,The cook is preparing a delicious meal in the ...,The chef is preparing a delicious breakfast in...,The chef is de-brief a delicious meal in the k...
3,3,A bright rainbow stretches across the sky afte...,1,A bright rain stretches across the sky after t...,A bright rainbow doesn't stretches across the ...,A bright rainbow stretches across the sky afte...,A bright anya stretches across the sky after t...,A bright rainbow stretches across the cloud af...,A bright rainbow bicep across the sky after th...
4,4,The students are listening attentively during ...,1,The lecture are listening attentively during t...,The students are not listening attentively dur...,The students are listening attentively during ...,The students are listening attentively during ...,The students are listening attentively during ...,The students are talk attentively during the l...


In [26]:
df.to_csv('/Users/raunakpandey/Documents/programming/projects/augmentation/Contradictions.csv') 